In [1]:
%config IPCompleter.greedy=True
from IPython.display import IFrame

import os
import time

import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from sklearn import preprocessing as prep
import sklearn.model_selection as model_selection
from sklearn import cluster
from sklearn import mixture

import tensorflow as tf
from tensorflow import keras
import keras.backend as K

import h5py
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
# strategy = tf.distribute.MirroredStrategy()

In [4]:
print(len(ALL_IDS), len(REVEAL_IDS_S2), len(NOREVEAL_IDS))

11754 510 11754


In [24]:
# TEST_IDS = [map_id.split('.')[0] for map_id in sorted(os.listdir('00_Data/fMRI_test'))]
# TRAIN_IDS = [map_id.split('.')[0] for map_id in sorted(os.listdir('00_Data/fMRI_train'))]
# ALL_IDS = TRAIN_IDS + TEST_IDS
# # REVEAL_IDS_S2 = list(pd.read_csv('00_Data/reveal_ID_site2.csv', header=0, squeeze=True).values)
# REVEAL_IDS_S2 = np.genfromtxt('00_Data/reveal_ID_site2.csv', delimiter='\n')
# REVEAL_IDS_S2 = REVEAL_IDS_S2[1:]
# REVEAL_IDS_S2 = REVEAL_IDS_S2.astype(int)
# NOREVEAL_IDS = [i for i in ALL_IDS if i not in REVEAL_IDS_S2]

<ipython-input-24-10ffc12f77d3>:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  NOREVEAL_IDS = [i for i in ALL_IDS if i not in REVEAL_IDS_S2]


In [102]:
TEST_IDS = [map_id.split('.')[0] for map_id in sorted(os.listdir('00_Data/fMRI_test'))]
TRAIN_IDS = [map_id.split('.')[0] for map_id in sorted(os.listdir('00_Data/fMRI_train'))]
ALL_IDS = TRAIN_IDS + TEST_IDS
REVEAL_IDS_S2 = pd.read_csv('00_Data/reveal_ID_site2.csv', header=None, squeeze=True, dtype=str)
# REVEAL_IDS_S2 = pd.read_csv('00_Data/reveal_ID_site2.csv', header=None, squeeze=True, dtype=np.int64)
NOREVEAL_IDS = [i for i in ALL_IDS if i not in REVEAL_IDS_S2.values]

In [103]:
len(NOREVEAL_IDS)

11244

In [63]:
data = pd.read_csv('00_Data/train_scores_full.csv')
# data

In [7]:
def get_inputs(idx):
    #MRI inputs
    patient_SM = h5py.File('00_Data/fMRI_all/{0}.mat'.format(idx), mode='r')
    patient_SM = np.array(patient_SM.get('SM_feature'))
#     print(patient_SM.shape)
    k = 1
    ki_padding = 3
    
    arr_regions = []
    for i in range(patient_SM.shape[0]):
        sample_map = patient_SM[i,:,:,:]
        if k > 1:
            map_shape = sample_map.shape
            shape_pad = ((map_shape[0]//k + 1)*k - map_shape[0],
                         (map_shape[1]//k + 1)*k - map_shape[1],
                         (map_shape[2]//k + 1)*k - map_shape[2])

            npad = (((0 if shape_pad[0]%2==0 else shape_pad[0]//2), (shape_pad[0]//2 if shape_pad[0]%2==0 else shape_pad[0]//2+1)),    
                    ((0 if shape_pad[1]%2==0 else shape_pad[0]//2), (shape_pad[1]//2 if shape_pad[1]%2==0 else shape_pad[1]//2+1)),    
                    ((0 if shape_pad[2]%2==0 else shape_pad[0]//2), (shape_pad[2]//2 if shape_pad[2]%2==0 else shape_pad[2]//2+1)))

            sample_map_padded = np.pad(sample_map, pad_width=npad, mode='constant', constant_values=0)

            sx = sample_map_padded.shape[0] / k
            sy = sample_map_padded.shape[1] / k
            sz = sample_map_padded.shape[2] / k
            for kz in range(k):
                for ky in range(k):
                    for kx in range(k):
                        ki_region = sample_map_padded[int(kx*sx): int(kx*sx + sx - 1), 
                                                     int(ky*sy): int(ky*sy + sy - 1), 
                                                     int(kz*sz): int(kz*sz + sz - 1)]
                        #padding i-th region by 3 pixels
                        ki_region_padded = np.pad(ki_region, pad_width=ki_padding, mode='constant', constant_values=0)
                        arr_regions.append(ki_region_padded)
        else:
            map_shape = sample_map.shape
            shape_pad = ((map_shape[0]//2 + 1)*2 - map_shape[0],
                         (map_shape[1]//2 + 1)*2 - map_shape[1],
                         (map_shape[2]//2 + 1)*2 - map_shape[2])

            npad = (((0 if shape_pad[0]%2==0 else shape_pad[0]//2+1), (0 if shape_pad[0]%2==0 else shape_pad[0]//2+1)),    
                    ((0 if shape_pad[1]%2==0 else shape_pad[0]//2+1), (0 if shape_pad[1]%2==0 else shape_pad[1]//2+1)),    
                    ((0 if shape_pad[2]%2==0 else shape_pad[0]//2+1), (0 if shape_pad[2]%2==0 else shape_pad[2]//2+1)))

            sample_map_padded = np.pad(sample_map, pad_width=npad, mode='constant', constant_values=0)
            
#             sample_map_padded = np.pad(sample_map, pad_width=ki_padding, mode='constant', constant_values=0)
            arr_regions.append(sample_map_padded)
            
    X_mri = np.stack(arr_regions, axis=3)
#     print(X_mri.shape)
    return X_mri, idx

In [8]:
# get_inputs('10002')[0].shape

In [9]:
def new_py_function(func, inp, Tout, name=None):
    
    def wrapped_func(*flat_inp):
        reconstructed_inp = tf.nest.pack_sequence_as(inp, flat_inp,
                                                     expand_composites=True)
        out = func(*reconstructed_inp)
        return tf.nest.flatten(out, expand_composites=True)
    
    flat_Tout = tf.nest.flatten(Tout, expand_composites=True)
    flat_out = tf.py_function(func=wrapped_func, 
                              inp=tf.nest.flatten(inp, expand_composites=True),
                              Tout=[_tensor_spec_to_dtype(v) for v in flat_Tout],
                              name=name)
    spec_out = tf.nest.map_structure(_dtype_to_tensor_spec, Tout, expand_composites=True)
    out = tf.nest.pack_sequence_as(spec_out, flat_out, expand_composites=True)
    return out

def _dtype_to_tensor_spec(v):
    return tf.TensorSpec(None, v) if isinstance(v, tf.dtypes.DType) else v

def _tensor_spec_to_dtype(v):
    return v.dtype if isinstance(v, tf.TensorSpec) else v

In [10]:
def get_dataset(data, batch_size):
    data = np.array([int(i) for i in data])
    data = tf.data.Dataset.from_tensor_slices(data)
    data = data.shuffle(buffer_size=12000, seed=30, reshuffle_each_iteration=True)
    
    data = data.map(lambda idx: new_py_function(get_inputs, inp=[idx], 
                                                    Tout=(tf.TensorSpec(shape=(None, 52, 66, 56, 53), dtype=tf.dtypes.float64),
                                                          tf.int32), 
                                                name=None), 
                     num_parallel_calls=tf.data.experimental.AUTOTUNE, 
                     deterministic=False)
    data = data.batch(batch_size, drop_remainder=False)
    data = data.prefetch(tf.data.experimental.AUTOTUNE)
    return data

In [11]:
batch_size = 8

ds_train = get_dataset(ALL_IDS, batch_size)
ds_reveal_s2 = get_dataset(REVEAL_IDS_S2, batch_size)
ds_noreveal = get_dataset(NOREVEAL_IDS, batch_size)

In [12]:
# for i in ds_train.take(1):
#     pass

In [13]:
INPUT_SHAPE_mri = (52, 66, 56, 53)

In [14]:
def create_model(input_shape, filters=[32, 16, 8, 2]):
    
    #============================================================================
    # ENCODER
    #============================================================================
    inputs_mri = keras.layers.Input(shape=INPUT_SHAPE_mri, name='inpupt_mri')

    # convolution block #1
    x = keras.layers.Conv3D(filters[0], kernel_size=(3, 3, 3), strides=(1,1,1), padding='same',
                                  kernel_initializer=keras.initializers.he_normal(seed=30),
                                  bias_initializer=keras.initializers.Constant(0.01))(inputs_mri)
    x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
    x = keras.layers.Conv3D(filters[0], kernel_size=(3, 3, 3), strides=(1,1,1), padding='same',
                                  kernel_initializer=keras.initializers.he_normal(seed=30),
                                  bias_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
    x = keras.layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2,2,2))(x)
#     x, p1_idx = tf.nn.max_pool_with_argmax(x, ksize=[2], strides=[2], padding='SAME', name="p1")
    x = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                                              beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros',
                                              moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
                                              beta_constraint=None, gamma_constraint=None)(x)

    # convolution block #2
    x = keras.layers.Conv3D(filters[1], kernel_size=(3, 3, 3), strides=(1,1,1), padding='same',
                                  kernel_initializer=keras.initializers.he_normal(seed=30),
                                  bias_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
    x = keras.layers.Conv3D(filters[1], kernel_size=(3, 3, 3), strides=(1,1,1), padding='same',
                                  kernel_initializer=keras.initializers.he_normal(seed=30),
                                  bias_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
    x = keras.layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2,2,2))(x)
    x = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                                              beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros',
                                              moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
                                              beta_constraint=None, gamma_constraint=None)(x)

    # convolution block #3
    x = keras.layers.Conv3D(filters[2], kernel_size=(3, 3, 3), strides=(1,1,1), padding='same',
                                  kernel_initializer=keras.initializers.he_normal(seed=30),
                                  bias_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
    x = keras.layers.Conv3D(filters[2], kernel_size=(3, 3, 3), strides=(1,1,1), padding='same',
                                  kernel_initializer=keras.initializers.he_normal(seed=30),
                                  bias_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
    x = keras.layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2,2,2))(x)
    x = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                                              beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros',
                                              moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
                                              beta_constraint=None, gamma_constraint=None)(x)

    # convolution block #4
#     x = keras.layers.Conv3D(filters[3], kernel_size=(3, 3, 3), strides=(1,1,1), padding='same',
#                                   kernel_initializer=keras.initializers.he_normal(seed=30),
#                                   bias_initializer=keras.initializers.Constant(0.01))(x)
#     x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
#     x = keras.layers.Conv3D(filters[3], kernel_size=(3, 3, 3), strides=(1,1,1), padding='same',
#                                   kernel_initializer=keras.initializers.he_normal(seed=30),
#                                   bias_initializer=keras.initializers.Constant(0.01))(x)
#     x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
#     x = keras.layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2,2,2))(x)
#     x = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
#                                               beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros',
#                                               moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
#                                               beta_constraint=None, gamma_constraint=None)(x)
    

    flatten = keras.layers.Flatten(data_format='channels_last')(x)

    encoded = keras.layers.Dense(2,
                               kernel_initializer=keras.initializers.he_normal(seed=30),
                               bias_initializer=keras.initializers.Constant(5.))(flatten)
    encoded = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.5))(encoded)

    
    #============================================================================
    # DECODER
    #============================================================================
    x = keras.layers.Dense(filters[2]*int(input_shape[0]/8)*int(input_shape[1]/8)*int(input_shape[2]/8),
                           kernel_initializer=keras.initializers.he_normal(seed=30),
                           bias_initializer=keras.initializers.Constant(5.))(encoded)
    
    x = keras.layers.Reshape((int(input_shape[0]/8), int(input_shape[1]/8), int(input_shape[2]/8), filters[2]))(x)
    
    # convolution block #4
#     x = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(x)
#     x = tf.keras.layers.Conv3DTranspose(filters[2], kernel_size=(1, 1, 2), strides=(1,1,1), padding='valid',
#                                         kernel_initializer=keras.initializers.he_normal(seed=30),
#                                         bias_initializer=keras.initializers.Constant(0.01))(x)
#     x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
    
    # convolution block #3
    x = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(x)
    x = tf.keras.layers.Conv3DTranspose(filters[2], kernel_size=(2, 1, 1), strides=(1,1,1), padding='valid',
                                        kernel_initializer=keras.initializers.he_normal(seed=30),
                                        bias_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.Conv3DTranspose(filters[2], kernel_size=(2, 1, 1), strides=(1,1,1), padding='same',
                                        kernel_initializer=keras.initializers.he_normal(seed=30),
                                        bias_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
    
    # convolution block #2
    x = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(x)
    x = tf.keras.layers.Conv3DTranspose(filters[1], kernel_size=(1, 2, 1), strides=(1,1,1), padding='valid',
                                        kernel_initializer=keras.initializers.he_normal(seed=30),
                                        bias_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.Conv3DTranspose(filters[1], kernel_size=(1, 2, 1), strides=(1,1,1), padding='same',
                                        kernel_initializer=keras.initializers.he_normal(seed=30),
                                        bias_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
    
    # convolution block #1
    x = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(x)
    x = tf.keras.layers.Conv3DTranspose(filters[0], kernel_size=(1, 1, 1), strides=(1,1,1), padding='valid',
                                        kernel_initializer=keras.initializers.he_normal(seed=30),
                                        bias_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.Conv3DTranspose(filters[0], kernel_size=(1, 1, 1), strides=(1,1,1), padding='same',
                                        kernel_initializer=keras.initializers.he_normal(seed=30),
                                        bias_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.Conv3DTranspose(input_shape[3], kernel_size=(1, 1, 1), strides=(1,1,1), padding='valid',
                                        kernel_initializer=keras.initializers.he_normal(seed=30),
                                        bias_initializer=keras.initializers.Constant(0.01))(x)
    x = tf.keras.layers.PReLU(alpha_initializer=keras.initializers.Constant(0.01))(x)
    
    decoded = x
    
    #============================================================================
    # COMPILE
    #============================================================================
    autoencoder = keras.Model(inputs=inputs_mri, outputs=decoded, name='autoencoder')
    encoder = keras.Model(inputs=inputs_mri, outputs=encoded, name='encoder')

    optim = tf.keras.optimizers.Adadelta(learning_rate=0.001, rho=0.95)

#     METRICS = [keras.metrics.RootMeanSquaredError(name='rmse')]

#     autoencoder.compile(loss='mse', metrics=METRICS, optimizer=optim)
    autoencoder.compile(loss='mse', optimizer=optim)
    return autoencoder, encoder

In [15]:
autoencoder, encoder = create_model(INPUT_SHAPE_mri)

In [16]:
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inpupt_mri (InputLayer)      [(None, 52, 66, 56, 53)]  0         
_________________________________________________________________
conv3d (Conv3D)              (None, 52, 66, 56, 32)    45824     
_________________________________________________________________
p_re_lu (PReLU)              (None, 52, 66, 56, 32)    6150144   
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 52, 66, 56, 32)    27680     
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 52, 66, 56, 32)    6150144   
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 26, 33, 28, 32)    0         
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 33, 28, 32)    

In [17]:
autoencoder.load_weights('./99_Training_checkpoints/mri_clustering/run_02/model_weights_02.h5')

In [19]:
enc_preds = {}
with tqdm(total=len(ALL_IDS)) as pbar:
    for i in ALL_IDS:
        x, _ = get_inputs(i)
        x = x.reshape(1,52, 66, 56, 53)
        preds = encoder.predict(x, batch_size=1, verbose=0)
        enc_preds[i] = preds[0]
        pbar.update(1)
enc_preds = pd.DataFrame(enc_preds).T
enc_preds.to_csv('00_Data/encoder_preds.csv', index=True, header=False)

100%|██████████████████████████████████████████████████████████████████████████| 11754/11754 [1:41:46<00:00,  1.92it/s]


In [42]:
enc_preds.head()

,0,1
10001,0.240295,-0.394146
10002,-0.022644,-0.465305
10004,-0.099391,-0.320526
10005,0.452530,0.073141
10007,-0.091789,-0.036786


In [67]:
enc_preds_s2 = enc_preds.loc[REVEAL_IDS_S2]
enc_preds_na = enc_preds.loc[NOREVEAL_IDS]

In [68]:
y_reveal_s2_enc = enc_preds_s2.values
y_noreveal_enc = enc_preds_na.values

y_reveal_s2_enc_mean = np.mean(y_reveal_s2_enc, axis=0)
y_noreveal_enc_mean = np.mean(y_noreveal_enc, axis=0)

In [69]:
# y_reveal_s2_enc = np.genfromtxt('y_reveal_s2_enc.csv', delimiter=',')
# y_noreveal_enc = np.genfromtxt('y_noreveal_enc.csv', delimiter=',')

# y_reveal_s2_enc_mean = np.genfromtxt('y_reveal_s2_enc_mean.csv', delimiter=',')
# y_noreveal_enc_mean = np.genfromtxt('y_noreveal_enc_mean.csv', delimiter=',')

In [70]:
y_reveal_s2_enc_mean

array([-0.14742053, -0.07687217], dtype=float32)

In [71]:
y_noreveal_enc_mean

array([-0.10948468, -0.10004541], dtype=float32)

In [72]:
print(y_reveal_s2_enc.shape, y_noreveal_enc.shape)

(510, 2) (11244, 2)


In [73]:
y_all = np.append(y_reveal_s2_enc, y_noreveal_enc, axis=0)

In [77]:
y_all.shape

(11754, 2)

In [78]:
n_clusters = 2

In [79]:
class ClusteringLayer(keras.layers.Layer):
    """
    Clustering layer converts input sample (feature) to soft label.

    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: degrees of freedom parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = keras.layers.InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
#         print(input_dim)
        self.input_spec = keras.layers.InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.        
                 q_ij = 1/(1+dist(x_i, µ_j)^2), then normalize it.
                 q_ij can be interpreted as the probability of assigning sample i to cluster j.
                 (i.e., a soft assignment)
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
#         q = 1 / (1 + tf.norm(inputs - self.clusters) / self.alpha) ** (self.alpha + 1) / 2
#         q = q / tf.math.reduce_sum(q, axis=)
#         print(self.clusters)
#         print(inputs)
#         print(K.expand_dims(inputs, axis=1))
#         print(K.expand_dims(inputs, axis=1) - self.clusters)
#         print(K.square(K.expand_dims(inputs, axis=1) - self.clusters))
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) # Make sure each sample's 10 values add up to 1.
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [80]:
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
# model = keras.Model(inputs=encoder.input, outputs=clustering_layer)
model = keras.Model(inputs=encoder.input, outputs=[clustering_layer, autoencoder.output])

In [81]:
[y_reveal_s2_enc_mean, y_noreveal_enc_mean]

[array([-0.14742053, -0.07687217], dtype=float32),
 array([-0.10948468, -0.10004541], dtype=float32)]

In [82]:
kmeans = cluster.KMeans(n_clusters=n_clusters, init=np.array([y_reveal_s2_enc_mean, y_noreveal_enc_mean]))
# kmeans = cluster.KMeans(n_clusters=n_clusters)
y_pred_km = kmeans.fit_predict(y_all)
kmeans.cluster_centers_
# kmeans.labels_

c:\00_data\python38\lib\site-packages\sklearn\cluster\_kmeans.py:1008: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return self.fit(X, sample_weight=sample_weight).labels_


array([[-0.2764053 , -0.22242014],
       [ 0.192379  ,  0.12753388]], dtype=float32)

In [83]:
# array([[-0.27640564, -0.22242   ],
#        [ 0.19237857,  0.12753383]])

In [84]:
gm = mixture.GaussianMixture(n_components=2, 
                             covariance_type='full', 
                             tol=0.0001, 
                             reg_covar=1e-06, 
                             max_iter=2000, 
                             n_init=1, 
                             init_params='kmeans', 
                             weights_init=[1-len(y_reveal_s2_enc)/len(y_all),
                                           1-len(y_noreveal_enc)/len(y_all)], 
                             means_init=[y_reveal_s2_enc_mean, y_noreveal_enc_mean], 
                             precisions_init=None, 
                             random_state=30, 
                             verbose=2, 
                             verbose_interval=10)
y_pred_gm = gm.fit_predict(y_all)
gm.means_

Initialization 0
  Iteration 10	 time lapse 0.04987s	 ll change 0.00101
  Iteration 20	 time lapse 0.02493s	 ll change 0.00020
Initialization converged: True	 time lapse 0.08876s	 ll -0.59734


array([[-0.20951488, -0.20186344],
       [ 0.13619564,  0.15944615]])

In [85]:
# array([[ 0.04507672,  0.05673662],
#        [-0.22917223, -0.21675597]])

In [86]:
len(y_pred_gm[np.where(y_pred_gm==0)])

8903

In [87]:
len(y_pred_gm[np.where(y_pred_gm==1)])

2851

In [88]:
init_centers_km = kmeans.cluster_centers_
init_centers_gm = gm.means_

In [89]:
y_pred_s2 = gm.predict(y_reveal_s2_enc)
y0 = y_pred_s2[np.where(y_pred_s2==0)]
y1 = y_pred_s2[np.where(y_pred_s2==1)]

print(y0.shape,y1.shape)

(384,) (126,)


In [90]:
y_pred_s2 = kmeans.predict(y_reveal_s2_enc)
y0 = y_pred_s2[np.where(y_pred_s2==0)]
y1 = y_pred_s2[np.where(y_pred_s2==1)]

print(y0.shape,y1.shape)

(339,) (171,)


In [91]:
model.get_layer(name='clustering').set_weights([init_centers_gm])
# model.get_layer(name='clustering').set_weights([init_centers_km])
optim = tf.keras.optimizers.Adadelta(learning_rate=0.001, rho=0.95)
# optim = tf.keras.optimizers.SGD(learning_rate=0.00001, momentum=0.9)
# model.compile(loss='kld', optimizer=optim)
model.compile(loss=['kld', 'mse'], loss_weights=[0.1, 1], optimizer=optim)

In [92]:
y_pred_last = np.copy(y_pred_gm)

In [93]:
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
#     print(weight.shape)
    return (weight.T / weight.sum(1)).T

In [94]:
loss = 0
index = 0
# maxiter = 1470
update_interval = 489
# index_array = np.arange(x.shape[0])

In [95]:
tol = 0.001 # tolerance threshold to stop training

In [96]:
# q = pd.read_csv('00_Data/q_distrib/q_1_1467.csv', index_col=0, header=None)
# y_pred = np.argmax(q.values, axis=1)
# #             print(q.values)
# #             print(q.loc[REVEAL_IDS_S2[2]])
# y_pred_s2 = np.argmax(q.loc[REVEAL_IDS_S2].values, axis=1)
# pos_rate = len(y_pred_s2[np.where(y_pred_s2==1)]) / len(y_pred_s2)
# neg_rate = len(y_pred_s2[np.where(y_pred_s2==0)])/len(y_pred_s2)
# print(pos_rate,neg_rate)

In [109]:
# IDS_S2 = pd.read_csv('00_Data/reveal_ID_site2.csv', header=None, dtype=np.int64)

In [112]:
# IDS_S2.to_csv('00_Data/reveal_ID_site2_int.csv', index=True, header=False)

In [114]:
# with tf.device('/GPU:0'):
epochs = 200
losses = []

for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    step_losses = []
    po_neg_rates = []
    for step, (x_batch, idx) in enumerate(ds_train):
        if step % update_interval == 0:
            if os.path.exists('00_Data/q_distrib/q_{0}_{1}.csv'.format(epoch, step)):
                q = pd.read_csv('00_Data/q_distrib/q_{0}_{1}.csv'.format(epoch, step), index_col=0, header=None)
                if os.path.exists('00_Data/p_distrib/p_{0}_{1}.csv'.format(epoch, step)):
                    p = pd.read_csv('00_Data/p_distrib/p_{0}_{1}.csv'.format(epoch, step), index_col=0, header=None)
                else:
                    p = target_distribution(q.values)  # update the auxiliary target distribution p
                    p = pd.DataFrame(p, index=q.index.to_list())
                    p.to_csv('00_Data/p_distrib/p_{0}_{1}.csv'.format(epoch, step), index=True, header=False)
                    p = pd.read_csv('00_Data/p_distrib/p_{0}_{1}.csv'.format(epoch, step), index_col=0, header=None)
#             if step==0:
#                 q = pd.read_csv('00_Data/q_distrib/q_0_0.csv', index_col=0, header=None)
#                 p = pd.read_csv('00_Data/p_distrib/p_0_0.csv', index_col=0, header=None)
            else:
                print('Update target distribution epoch {0} step {1}'.format(epoch, step))
                q = {}
                with tqdm(total=len(ALL_IDS)) as pbar:
                    for i in ALL_IDS:
                        x, _ = get_inputs(i)
                        x = x.reshape(1,52, 66, 56, 53)
                        preds, _ = model.predict(x, batch_size=1, verbose=0)
                        q[i] = preds[0]
                        pbar.update(1)
                q = pd.DataFrame(q).T
                q.to_csv('00_Data/q_distrib/q_{0}_{1}.csv'.format(epoch, step), index=True, header=False)
                q = pd.read_csv('00_Data/q_distrib/q_{0}_{1}.csv'.format(epoch, step), index_col=0, header=None)
                p = target_distribution(q.values)  # update the auxiliary target distribution p
                p = pd.DataFrame(p, index=q.index.to_list())
                p.to_csv('00_Data/p_distrib/p_{0}_{1}.csv'.format(epoch, step), index=True, header=False)
                p = pd.read_csv('00_Data/p_distrib/p_{0}_{1}.csv'.format(epoch, step), index_col=0, header=None)

            # evaluate the clustering performance
#             display(q)
#             print(q.index)
            IDS_S2 = pd.read_csv('00_Data/reveal_ID_site2_int.csv', index_col=0, header=None, squeeze=True, dtype=np.int64)
            y_pred = np.argmax(q.values, axis=1)
            y_pred_s2 = np.argmax(q.loc[IDS_S2].values, axis=1)
            pos_rate = len(y_pred_s2[np.where(y_pred_s2==1)]) / len(y_pred_s2)
            neg_rate = len(y_pred_s2[np.where(y_pred_s2==0)])/len(y_pred_s2)
            po_neg_rates.append([pos_rate, neg_rate])
            print('Pos. rate:{0} Neg. rate:{1}'.format(pos_rate, neg_rate))

            # check stop criterion
            delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
            print('delta_label ', delta_label)
            y_pred_last = np.copy(y_pred)
            if step > 0 and delta_label < tol:
                print('delta_label ', delta_label, '< tol ', tol)
                print('Reached tolerance threshold. Stopping training.')
                break
                
        y_probs = p.loc[idx]
        y_probs = y_probs.values
        loss = model.train_on_batch(x=x_batch, y=[y_probs, x_batch])
        step_losses.append(loss)
        print('Step {0}, loss = {1}'.format(step, loss))
        
    epoch_avloss = np.mean(np.array(step_losses), axis=0)
    print('Epoch {0}, loss = {1}'.format(epoch, epoch_avloss))
    losses.append(epoch_avloss)
    model.save_weights('99_Training_checkpoints/mri_clustering/run_05/model_weights_chpt_{0}.h5'.format(epoch))

np.savetxt('99_Logs/mri_clustering/run_05/pos_neg_rates.csv', np.array(po_neg_rates), delimiter=',')
np.savetxt('99_Logs/mri_clustering/run_05/losses.csv', np.array(losses), delimiter=',')
model.save_weights('99_Training_checkpoints/mri_clustering/run_05/model_weights_final.h5')


Start of epoch 0
Pos. rate:0.5941176470588235 Neg. rate:0.40588235294117647
delta_label  0.9646928705121661
Step 0, loss = [0.3113613724708557, 0.05623418837785721, 0.3057379424571991]
Step 1, loss = [0.30847546458244324, 0.048053279519081116, 0.30367013812065125]
Step 2, loss = [0.2746799886226654, 0.032676372677087784, 0.27141234278678894]
Step 3, loss = [0.30908530950546265, 0.04608887806534767, 0.30447641015052795]
Step 4, loss = [0.3121620714664459, 0.03367875516414642, 0.30879420042037964]
Step 5, loss = [0.30222317576408386, 0.040087565779685974, 0.2982144057750702]
Step 6, loss = [0.30569857358932495, 0.03683668375015259, 0.30201491713523865]
Step 7, loss = [0.3000527322292328, 0.031579673290252686, 0.29689475893974304]
Step 8, loss = [0.3159763813018799, 0.03973408043384552, 0.3120029866695404]
Step 9, loss = [0.30248501896858215, 0.05552027374505997, 0.29693299531936646]
Step 10, loss = [0.3048131763935089, 0.035948753356933594, 0.30121830105781555]
Step 11, loss = [0.310885

  0%|                                                                                        | 0/11754 [00:00<?, ?it/s]

Update target distribution epoch 1 step 0


100%|██████████████████████████████████████████████████████████████████████████| 11754/11754 [1:51:23<00:00,  1.76it/s]


Pos. rate:0.38235294117647056 Neg. rate:0.6176470588235294
delta_label  0.0064658839543985025
Step 0, loss = [0.2969430983066559, 0.0020694027189165354, 0.2967361509799957]
Step 1, loss = [0.3024516999721527, 0.00957782007753849, 0.30149391293525696]
Step 2, loss = [0.30714958906173706, 0.00803665816783905, 0.3063459098339081]
Step 3, loss = [0.3078710734844208, 0.0054796962067484856, 0.3073230981826782]
Step 4, loss = [0.29297882318496704, 0.006692391820251942, 0.29230958223342896]
Step 5, loss = [0.2881767451763153, 0.0051764328964054585, 0.2876591086387634]
Step 6, loss = [0.2810010612010956, 0.012900471687316895, 0.2797110080718994]
Step 7, loss = [0.29913222789764404, 0.006043660454452038, 0.29852786660194397]
Step 8, loss = [0.2936554253101349, 0.006484492216259241, 0.2930069863796234]
Step 9, loss = [0.29942086338996887, 0.003038830356672406, 0.29911696910858154]
Step 10, loss = [0.3152118921279907, 0.0013570496812462807, 0.31507617235183716]
Step 11, loss = [0.28628525137901306

...

Pos. rate:0.4294117647058823 Neg. rate:0.5705882352941176
delta_label  0.003998638761272758
Step 489, loss = [0.29302090406417847, 0.007259399630129337, 0.2922949492931366]
Step 490, loss = [0.2820057272911072, 0.004737743176519871, 0.28153195977211]
Step 491, loss = [0.3095548152923584, 0.006209831219166517, 0.308933824300766]
Step 492, loss = [0.29890018701553345, 0.005747778341174126, 0.29832541942596436]
Step 493, loss = [0.3166971802711487, 0.006378655321896076, 0.31605932116508484]
Step 494, loss = [0.2984694540500641, 0.008106589317321777, 0.2976588010787964]
Step 495, loss = [0.27958306670188904, 0.009048951789736748, 0.27867817878723145]
Step 496, loss = [0.30459704995155334, 0.012540893629193306, 0.3033429682254791]
Step 497, loss = [0.30701199173927307, 0.004436110612004995, 0.3065683841705322]
Step 498, loss = [0.31263989210128784, 0.008075371384620667, 0.31183236837387085]
Step 499, loss = [0.28291159868240356, 0.009492738172411919, 0.2819623351097107]
Step 500, loss = [0.

  0%|                                                                                        | 0/11754 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████| 11754/11754 [1:52:18<00:00,  1.74it/s]


Pos. rate:0.44509803921568625 Neg. rate:0.5549019607843138
delta_label  0.008677896886166412
Step 978, loss = [0.3048257529735565, 0.012576408684253693, 0.30356812477111816]
Step 979, loss = [0.30603262782096863, 0.004208499565720558, 0.305611789226532]
Step 980, loss = [0.3070702850818634, 0.006862865295261145, 0.30638399720191956]
Step 981, loss = [0.29435864090919495, 0.01050366647541523, 0.293308287858963]
Step 982, loss = [0.3121262192726135, 0.004766902420669794, 0.31164953112602234]
Step 983, loss = [0.30701321363449097, 0.010291334241628647, 0.3059840798377991]
Step 984, loss = [0.2967810332775116, 0.004121347330510616, 0.29636889696121216]
Step 985, loss = [0.30495697259902954, 0.010938793420791626, 0.3038631081581116]
Step 986, loss = [0.29762759804725647, 0.008408615365624428, 0.29678672552108765]


ResourceExhaustedError: Failed to allocate memory for the batch of component 0

In [ ]:
# epochs = 2
# for epoch in range(epochs):
#     print("\nStart of epoch %d" % (epoch,))

#     # Iterate over the batches of the dataset.
#     for step, (x_batch_train, y_batch_train) in enumerate(ds_train):

#         # Open a GradientTape to record the operations run
#         # during the forward pass, which enables autodifferentiation.
#         with tf.GradientTape() as tape:

#             # Run the forward pass of the layer.
#             # The operations that the layer applies
#             # to its inputs are going to be recorded
#             # on the GradientTape.
#             logits = model(x_batch_train, training=True)  # Logits for this minibatch

#             # Compute the loss value for this minibatch.
#             loss_value = loss_fn(y_batch_train, logits)

#         # Use the gradient tape to automatically retrieve
#         # the gradients of the trainable variables with respect to the loss.
#         grads = tape.gradient(loss_value, model.trainable_weights)

#         # Run one step of gradient descent by updating
#         # the value of the variables to minimize the loss.
#         optimizer.apply_gradients(zip(grads, model.trainable_weights))

#         # Log every 200 batches.
#         if step % 200 == 0:
#             print(
#                 "Training loss (for one batch) at step %d: %.4f"
#                 % (step, float(loss_value))
#             )
#             print("Seen so far: %s samples" % ((step + 1) * 64))